### doc_topics
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E TÓPICOS NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.

In [1]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use 

In [2]:
outputs = "../outputs/"

In [3]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Seleciona textos da base mysql para fazer modelagem de tópicos
NOTA: DEIXEI EM RAW PORQUE ESSA EXTRAÇÃO TORNA O CÓDIGO MUITO LENTO, MAS DEVE SER FEITO DESSA FORMA NO CPU DO RENATO
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
texts = []
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()

with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    captura documentos da tabela docs para posterior modelagem de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    cur.execute("SELECT * FROM CPDOC_AS.docs WHERE main_language = 1") # seleciona apenas textos majoritariamente em português    
    data = cur.fetchall()
    numrows = cur.rowcount
    percentil = numrows/100
    
    for row in data:
        count += 1
        if row is None: break

        if count % percentil == 0: 
            clear_output()
            print(int(count/percentil),'% done')

        text =  row[1]
        text = text.split()
        symbols = [x for x in string.punctuation]
        text = [p for p in text if p not in symbols]
        text = [p.strip(string.punctuation) for p in text]
        text = [p for p in text if not p.isdigit()]
        text = [p for p in text if len(p)>1]
        texts.append(text)
        
        #if count == 3: break #amostra de apenas 3 linhas. No computador do Renato, não usar o break.

········
········


In [ ]:
# load nltk's English stopwords as variable called 'stopwords'

additional_words = ['mr','one', 'two', 'three', 'four', 
                    'five', 'um', 'dois', 'três', 'quatro', 
                    'cinco', 'janeiro', 'fevereiro', 'março', 
                    'abril', 'maio', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 
                    'january', 'february', 'march', 'april', 'may', 
                    'june', 'july', 'august', 'september', 
                    'october', 'november', 'december', 'países', 
                    'ser', 'país', 'ainda', 'milhões', 'maior', 
                    'anos', 'grande', 'apenas', 'outros', 'pode', 
                    'parte', 'partes', 'item', 'vossa', 'nota', 
                    'havia', 'pt', 'vg', 'ptvg', 'eh', 'nr', 'hrs', 
                    'pais', 'parte', 'hoje', 'brasemb', 'ontem', 
                    'dia', 'countries', 'would', 'new', 'also', 
                    'must', 'draft', 'shall', 'item', 'page', 
                    'th', 'anos', 'ii', 'dias', 'poderá', 'caso', 
                    'casos', 'qualquer', 'ano', 'mil', 'pessoas', 
                    'único', 'única', 'únicos', 'únicas', 'índice', 
                    'expedido', 'co', 'mm', 'er', 'via', 'ww', 'ra', 
                    'ia', 'ca', 'nu', 'wa', 'aa', 'ms', 'dc', 'mmm', 'pa']

stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            additional_words

stopwords = list(set(stopwords))

### Latent Dirichlet Allocation

In [4]:
#%time texts = [[word for word in text] for text in texts]
%time texts = [[word for word in text if word not in stopwords] for text in textos]

Wall time: 0 ns


In [5]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

218


In [6]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
#print(dictionary.token2id['']) #examina o id a partir do token
#print(dictionary.id2token[0]) #examina o token a partir do id
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [7]:
len(corpus)

3

In [8]:
%time lda = models.LdaModel(corpus, num_topics=50, id2word=dictionary, passes=10)

Wall time: 506 ms


In [9]:
lda.print_topics(-1, num_words=20)

[(0,
  '0.003*da + 0.003*do + 0.003*humanos + 0.003*no + 0.003*direitos + 0.003*que + 0.003*grupo + 0.003*comunicações + 0.003*brasil + 0.003*por + 0.003*as + 0.003*foram + 0.003*contra + 0.003*ao + 0.003*comissão + 0.003*membro + 0.003*se + 0.003*subcomissão + 0.003*na + 0.003*alegações'),
 (1,
  '0.003*sr + 0.003*vê-lo + 0.003*posterior + 0.003*passados + 0.003*caro + 0.003*sentido + 0.003*nenhuma + 0.003*durante + 0.003*ainda + 0.003*et + 0.003*mé + 0.003*uma + 0.003*local + 0.003*gestões + 0.003*aas + 0.003*disposición + 0.003*discriminação + 0.003*embaixada + 0.003*as + 0.003*con'),
 (2,
  '0.004*da + 0.004*do + 0.004*que + 0.004*no + 0.003*grupo + 0.003*por + 0.003*humanos + 0.003*direitos + 0.003*dos + 0.003*comissão + 0.003*as + 0.003*comunicações + 0.003*brasil + 0.003*genebra + 0.003*embaixador + 0.003*junto + 0.003*na + 0.003*américa + 0.003*se + 0.003*contra'),
 (3,
  '0.041*do + 0.038*da + 0.020*no + 0.020*grupo + 0.018*que + 0.018*humanos + 0.018*direitos + 0.015*dos + 0.

In [10]:
lda.show_topics(num_topics=-1)

[(0,
  '0.003*da + 0.003*do + 0.003*humanos + 0.003*no + 0.003*direitos + 0.003*que + 0.003*grupo + 0.003*comunicações + 0.003*brasil + 0.003*por'),
 (1,
  '0.003*sr + 0.003*embaixada + 0.003*discriminação + 0.003*disposición + 0.003*aas + 0.003*gestões + 0.003*local + 0.003*uma + 0.003*mé + 0.003*et'),
 (2,
  '0.004*da + 0.004*do + 0.004*que + 0.004*no + 0.003*grupo + 0.003*por + 0.003*humanos + 0.003*direitos + 0.003*dos + 0.003*comissão'),
 (3,
  '0.041*do + 0.038*da + 0.020*no + 0.020*grupo + 0.018*que + 0.018*humanos + 0.018*direitos + 0.015*dos + 0.015*por + 0.015*brasil'),
 (4,
  '0.003*sr + 0.003*embaixada + 0.003*discriminação + 0.003*disposición + 0.003*aas + 0.003*gestões + 0.003*local + 0.003*uma + 0.003*mé + 0.003*et'),
 (5,
  '0.003*sr + 0.003*embaixada + 0.003*discriminação + 0.003*disposición + 0.003*aas + 0.003*gestões + 0.003*local + 0.003*uma + 0.003*mé + 0.003*et'),
 (6,
  '0.003*sr + 0.003*embaixada + 0.003*discriminação + 0.003*disposición + 0.003*aas + 0.003*gest

In [11]:
topics_matrix = lda.show_topics(formatted=False, num_words=20, num_topics=-1)
topics_matrix[34]

(34,
 [('sr', 0.002673796791443851),
  ('vê-lo', 0.002673796791443851),
  ('posterior', 0.002673796791443851),
  ('passados', 0.002673796791443851),
  ('caro', 0.002673796791443851),
  ('sentido', 0.002673796791443851),
  ('nenhuma', 0.002673796791443851),
  ('durante', 0.002673796791443851),
  ('ainda', 0.002673796791443851),
  ('et', 0.002673796791443851),
  ('mé', 0.002673796791443851),
  ('uma', 0.002673796791443851),
  ('local', 0.002673796791443851),
  ('gestões', 0.002673796791443851),
  ('aas', 0.002673796791443851),
  ('disposición', 0.002673796791443851),
  ('discriminação', 0.002673796791443851),
  ('embaixada', 0.002673796791443851),
  ('as', 0.002673796791443851),
  ('con', 0.002673796791443851)])

Gerando a distribuição de tópicos para um novo documento:

#### Gera a distribuição de tópicos para um novo documento
Alimenta a base mysql com dados de score (relação) entre tópicos e documentos

In [12]:
count = 0
#percentil = int(len(cursor.rowcount/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS topic_doc_lang_filter")
    cur.execute('''CREATE TABLE IF NOT EXISTS topic_doc_lang_filter 
               (doc_id VARCHAR(31), topic_id smallint(6), topic_score FLOAT
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT id,body FROM CPDOC_AS.docs")
    data = cur.fetchall()
    for row in data:
        count += 1 # for break event
        if row is None: break
        text =  row[1]
        text = text.split()
        text_bow = dictionary.doc2bow(text)
        print(lda[text_bow])
        score_list = lda[text_bow]
        doc_id = row[0]
        for score in score_list:
            topic_id = str(score[0])
            topic_score = str(score[1])
            query = "INSERT INTO topic_doc_lang_filter VALUES (%s, %s, %s)"
            cur.execute(query, (doc_id, topic_id, topic_score))
        print(doc_id, topic_id, topic_score)   
        cur.execute("ALTER TABLE CPDOC_AS.topic_doc_lang_filter ORDER BY topic_id ASC, topic_score DESC")
        
        #if count == 10: break #amostra de apenas 10 linhas

········
········
[(7, 0.9938749999999974)]
rb_1974.04.17_doc_I-66 7 0.993875
[(36, 0.97487179487179387)]
onu_1974.03.12_doc_II-49 36 0.974871794872
[(7, 0.50999999999999979)]
onu_1974.03.12_doc_V-2 7 0.51
[(3, 0.99634328358209268)]
be_1974.04.16_doc_VI-18 3 0.996343283582
[(3, 0.90858161141617577), (7, 0.049972685004809121), (36, 0.030515471020877497)]
be_1977.06.01_doc_II-21 36 0.0304864918981
[(3, 0.54800575380191874), (7, 0.25160963664082597), (36, 0.19663958963693487)]
ag_1974.03.13_doc_III-7 36 0.196650458542
[(3, 0.068000000000000005), (7, 0.14323339290505094), (36, 0.72609994042828274)]
ag_1973.11.20_doc_VI-75 36 0.725978313008
[(3, 0.75655064086215906), (7, 0.16344935913784245)]
d_1974.04.23_doc_XXXII-8 7 0.162419385224
[(3, 0.81177208746132345), (7, 0.15601219071071712), (36, 0.030049823210449585)]
d_1974.03.26_doc_XXXII-32 36 0.0300513188816
[(3, 0.81942653401341525), (7, 0.174253366815973)]
d_1974.03.26_doc_XXII-9 7 0.174253373038


#### Visualizando os tópicos com o PyLDAvis

In [ ]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_langfilter_output.html'))